## Model Performance for Radar Data

In [2]:
import pandas as pd
import numpy as np
import scipy as sp
import torch
import os.path as osp
import seaborn as sb
from matplotlib import pyplot as plt
import glob

In [3]:
base_dir = '/home/fiona/birdMigration/results/radar/testing'

### Load Results

In [ ]:
models = ['GBT', 'LocalMLP', 'LocalLSTM', 'GraphLSTM', 'BirdFlowGraphLSTM']
experiments1 = ['_not_transformed', '_not_transformed', '', '', '_root_transformed', '_not_recurrent_force_zeros']